# Install Dependencies
Install the required Python packages to run LangChain and interact with LLMs. This ensures all modules are available before building the app.

In [1]:
from IPython.core.debugger import prompt
!pip install -qU "langchain[google-genai]"

# Set Google API Key
Load the Google API key from the environment to authenticate our requests. This protects the key from being hard-coded into the notebook.

In [4]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

# Create an LLM
Initialize the Gemini language model using LangChain’s wrapper. This model will generate text based on the prompts we define.


In [6]:
from langchain.chat_models import init_chat_model


In [7]:
model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [8]:
model.invoke("Hello, world!")

AIMessage(content='Hello there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--307d33f1-c9b1-4f66-b82c-06caa5d37789-0', usage_metadata={'input_tokens': 4, 'output_tokens': 11, 'total_tokens': 15, 'input_token_details': {'cache_read': 0}})

# Promt Templates
Define reusable prompt templates that act as structured instructions for the LLM. This makes input formatting consistent and easier to manage.

In [9]:
from langchain.prompts import PromptTemplate

In [11]:
from langchain.prompts import PromptTemplate

summary_template = PromptTemplate(
    input_variables=["document_text"],
    template="""
You are a professional document summarizer.

Your task is to read the following block of text extracted from a document (such as a PDF) and generate a clear and concise summary in bullet points.

Instructions:
- Keep the summary under 150 words.
- Highlight only the most important ideas or facts.
- Avoid copying exact sentences from the original text.
- Use neutral, objective language.

Text to summarize:
{document_text}

Summary:
"""
)

# Create the Langchain Chain
Build a chain that combines the prompt and the LLM into a single callable object. This lets us run the model in a pipeline-friendly way.

In [12]:
from langchain.chains import LLMChain

In [15]:
chain = summary_template | model

# Testing out the chain
Pass sample text into the chain to verify that the prompt and LLM integration work correctly. This step confirms that the summarization or other logic behaves as expected.

In [14]:
text = """
Technology has become an essential part of our daily lives. From the moment we wake up to the time we go to sleep, we interact with some form of technology. Smartphones, computers, and smart devices help us communicate, work, study, and entertain ourselves more efficiently than ever before.

In the past, simple tasks like sending a letter or finding a recipe could take hours or even days. Today, these things can be done instantly with a few clicks or voice commands. Education has also changed — students can attend classes online, access global libraries, and use AI tools to improve their learning.

However, technology also brings challenges. People spend more time in front of screens, which can affect their health and reduce face-to-face interactions. Additionally, rapid technological change means that workers need to constantly update their skills to remain relevant.

In conclusion, technology offers many benefits, but it also requires us to adapt and use it responsibly.
"""

In [16]:
result = chain.invoke({'document_text':text})

In [18]:
result

AIMessage(content='Here is a summary of the provided text in bullet points:\n\n*   Technology is integral to modern life, enhancing communication, work, education, and entertainment.\n*   Technology streamlines tasks, offering speed and convenience compared to traditional methods.\n*   Education is transformed through online learning, digital resources, and AI tools.\n*   Increased screen time and reduced social interaction are potential negative impacts.\n*   Continuous upskilling is necessary for workers to adapt to rapid technological advancements.\n*   Responsible technology use and adaptation are crucial to maximize its benefits.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--6097caae-8cd4-45b8-992f-492da98ca46c-0', usage_metadata={'input_tokens': 285, 'output_tokens': 114, 'total_tokens': 399, 'input_token_details': {'cache_read': 0}